# CSILSP-AGG

In [1]:
import gurobipy as gb
from CSILSPGenerator import Generator

#### Decision variables:

$$
X_t = \begin{cases}
\text{ quantity of product to be produced/supplied in period t }
\end{cases}
$$

$$
I_t = \begin{cases}
\text{ inventory level at the end of period t }
\end{cases}
$$

$$
Y_t = \begin{cases}
1 \text{ if } X_t > 0\\
0 \text{ otherwise }
\end{cases}
$$

#### Parameters:

$$
T = \begin{cases}
\text{ number of periods in the horizon }
\end{cases}
$$

$$
d_t = \begin{cases}
\text{ Customer aggregate demand that must } \\ \text{ be delivered at the end of period t }
\end{cases}
$$

$$
h_t = \begin{cases}
\text{ Holding cost per unit of product in stock at the end of period t }
\end{cases}
$$

$$
p_t = \begin{cases}
\text{ Unit production/supplying cost in period t }
\end{cases}
$$

$$
s_t = \begin{cases}
\text{ Fixed setup cost in period t }
\end{cases}
$$

$$
C_t = \begin{cases}
\text{ Production/Supply capacity at period t }
\end{cases}
$$

#### CSILSP-AGG (aggregate model)

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T} (h_tI_t + p_tX_t + s_tY_t) &\\
\text{s.t.} \;\;\;\;\;&\\
I_t &= I_{t-1} + X_t - d_t \;\; , & \forall t \;\;\\
X_t &\leq \sum_{k=t}^{T} d_kY_t \;\; , & \forall t\;\;\\
X_t,I_t &\geq 0 \;\; , & \forall t\;\;\\
X_t & \leq C_tY_t \;\; , & \forall t\;\;\\
Y_t &\in \{0,1\} \;\; , & \forall t \;\;
\end{alignat}
$$

In [2]:
# generation parameters

if __name__ == '__main__':
    T = 8
    l_b = 10
    u_b = 20

In [3]:
# generate instance

if __name__ == '__main__' or __name__ == '__autoexec__':
    instance = Generator(T)
    instance.generate(a=l_b,b=u_b)

In [4]:
if __name__ == '__main__':
    print(instance.printData())

    d   C   h   p   s
T                    
0  15  16  17  10  12
1  19  20  18  20  15
2  19  30  17  11  19
3  11  15  13  10  16
4  15  26  12  11  19
5  10  15  10  12  13
6  10  15  13  20  18
7  11  13  19  10  16


In [5]:
# variable bindings

T = instance.get_T()
d = instance.get_d()
C = instance.get_C()
h = instance.get_h()
p = instance.get_p()
s = instance.get_s()

In [6]:
# model

csilsp_agg = gb.Model()

Academic license - for non-commercial use only


In [7]:
# decision variables

X = csilsp_agg.addVars(T,vtype=gb.GRB.CONTINUOUS,lb=0.0, name='X')

I = csilsp_agg.addVars(T,vtype=gb.GRB.CONTINUOUS,lb=0.0, name='I')

Y = csilsp_agg.addVars(T,vtype=gb.GRB.BINARY,name='Y')

csilsp_agg.update()
csilsp_agg.write('csilsp_agg.lp')

$$
\begin{alignat}{3}
& \min \sum_{t=1}^{T} (h_tI_t + p_tX_t + s_tY_t) &
\end{alignat}
$$

In [8]:
# objective function

csilsp_agg.setObjective(I.prod(h) + X.prod(p) + Y.prod(s),gb.GRB.MINIMIZE)

csilsp_agg.update()
csilsp_agg.write('csilsp_agg.lp')

$$
\begin{alignat}{3}
I_t &= I_{t-1} + X_t - d_t \;\; , & \forall t
\end{alignat}
$$

In [9]:
# constraints

csilsp_agg.addConstrs( ( I[t] == I[t-1] + X[t] - d[t] for t in range(1,T-1) ),name='constrI')

csilsp_agg.update()
csilsp_agg.write('csilsp_agg.lp')

In [10]:
# assumption: starting and ending inventories are null

csilsp_agg.addConstr(I[0] == 0,name='startInventoryNull')
csilsp_agg.addConstr(I[T-1] == 0,name='endInventoryNull')

csilsp_agg.update()
csilsp_agg.write('csilsp_agg.lp')

$$
\begin{alignat}{3}
X_t &\leq \sum_{k=t}^{T} d_kY_t \;\; , & \forall t
\end{alignat}
$$

In [11]:
csilsp_agg.addConstrs( ( X[t] <= (gb.quicksum(d[k] for k in range(t,T)))*Y[t] for t in range(T) ),name='constX')

csilsp_agg.update()
csilsp_agg.write('csilsp_agg.lp')

Utilizzare vincolo (6) o vincolo (7):

$$
\begin{alignat}{3}
X_t & \leq C_t \;\; , & \forall t \;\; (6)
\end{alignat}
$$

In [12]:
#csilsp_agg.addConstrs( ( X[t] <= C[t] for t in range(T) ),name='constrC')

#csilsp_agg.update()
#csilsp_agg.write('csilsp_agg.lp')

$$
\begin{alignat}{3}
X_t & \leq C_tY_t \;\; , & \forall t \;\; (7)
\end{alignat}
$$

In [13]:
csilsp_agg.addConstrs( ( X[t] <= C[t]*Y[t] for t in range(T) ),name='constrC')

csilsp_agg.update()
csilsp_agg.write('csilsp_agg.lp')

In [14]:
# solve model

csilsp_agg.optimize()

Optimize a model with 24 rows, 24 columns and 52 nonzeros
Variable types: 16 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve removed 16 rows and 12 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 19 nonzeros
Variable types: 8 continuous, 4 integer (4 binary)

Root relaxation: objective 1.267362e+03, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1267.36154    0    3          - 1267.36154      -     -    0s
H    0     0                    1284.0000000 1267.36154  1.30%     -    0s
     0     0     cutoff    0      1284.00000 1284.00000  0.00%     -    0s

Explored 1 nodes (9 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 1284 


In [15]:
if __name__ == '__main__':
    print("Soluzione ottima:",csilsp_agg.ObjVal)

Soluzione ottima: 1284.0


In [16]:
X

{0: <gurobi.Var X[0] (value 0.0)>,
 1: <gurobi.Var X[1] (value 19.0)>,
 2: <gurobi.Var X[2] (value 19.0)>,
 3: <gurobi.Var X[3] (value 11.0)>,
 4: <gurobi.Var X[4] (value 15.0)>,
 5: <gurobi.Var X[5] (value 10.0)>,
 6: <gurobi.Var X[6] (value 10.0)>,
 7: <gurobi.Var X[7] (value 0.0)>}

In [17]:
Y

{0: <gurobi.Var Y[0] (value 0.0)>,
 1: <gurobi.Var Y[1] (value 1.0)>,
 2: <gurobi.Var Y[2] (value 1.0)>,
 3: <gurobi.Var Y[3] (value 1.0)>,
 4: <gurobi.Var Y[4] (value 1.0)>,
 5: <gurobi.Var Y[5] (value 1.0)>,
 6: <gurobi.Var Y[6] (value 1.0)>,
 7: <gurobi.Var Y[7] (value 0.0)>}

In [18]:
I

{0: <gurobi.Var I[0] (value 0.0)>,
 1: <gurobi.Var I[1] (value 0.0)>,
 2: <gurobi.Var I[2] (value 0.0)>,
 3: <gurobi.Var I[3] (value 0.0)>,
 4: <gurobi.Var I[4] (value 0.0)>,
 5: <gurobi.Var I[5] (value 0.0)>,
 6: <gurobi.Var I[6] (value 0.0)>,
 7: <gurobi.Var I[7] (value 0.0)>}

In [19]:
# execution time
if __name__ == '__main__':
    print("Optimize time: %.5g sec" % csilsp_agg.Runtime)

Optimize time: 0.052839 sec


In [20]:
# Write results to file

results = "----------------------------------------------------------------" + "\n"
results += " Parameter T = %d " % (T) + "\n"
results += " Parameters d,h,p,s,C random generation from %d to %d " % (l_b,u_b) + "\n"
results += " Execution time = %.5g sec " % csilsp_agg.Runtime + "\n"
results += "----------------------------------------------------------------" + "\n\n\n"

In [21]:
with open("csilsp_agg_results.txt", "a") as write_file:
    write_file.write(results)